# En este cuadernillo espero terminar de corregir las tablas, para luego continuar con la creacion de las vistas

### Retonamando lo ultimo del cuaderno anterior, es que se creo una nueva funcion para cambiar el tipo de la columna de una tabla a entero. Partiremos desde ahi, para modificar el resto de las tablas.


In [ ]:
# Nombre de las tablas 
# 'customers', 'DCs', 
# 'detergents', 'orders', 'plants', 'routes_available', 'routes_endtoend'
# 'routes_segment', 'variable_costs'


In [1]:
# Conectar con base de datos
from raaraya1 import SQL
import pandas as pd
import psycopg2

usuario = SQL('curso_postgresql', 'Tester', '123456')


## Tabla Costumers

In [14]:
# revisemos la tabla

comando = 'select * from customers'
df = usuario.mostrar_tabla(comando)
df

,0,1,2
0,Customer001,-6.974.150.410,0.572.172.540
1,Customer002,-1.774.631.895,0.771.807.615
2,Customer003,6.741.158.818,7.906.606.020
3,Customer004,-7.191.233.417,-2.539.319.369
4,Customer005,6.844.101.913,-6.249.004.925
5,Customer006,-5.887.849.020,-2.207.052.049
6,Customer007,2.739.095.208,-4.938.065.519


In [15]:
# esta tabla tiene un error en la escritura de latitud y la longitud
## cambiar a numero decimal, luego dividir por 100000000, luego volver a cambiar los valores a str

### para hacer esto primero extraigamos las columas y corrijamoslas
columna2 = list(df[1][:])
columna3 = list(df[2][:])

### ahora tranformemos estos valores a numero entero
columna2 = [int(valor.replace('.', ''))/100000000 for valor in columna2]
columna3 = [int(valor.replace('.', ''))/100000000 for valor in columna3]

### ahora volvamos a cambiar los valore de la columna a formato de texto
columna2 = [str(valor) for valor in columna2]
columna3 = [str(valor) for valor in columna3]

### ahora arreglemos los valore en la tabla
nombre_tabla = 'customers'
nombre_columna2 = 'CustomerLocationLatitude'
nombre_columna3 = 'CustomerLocationLongitude'

print(columna2)
print(columna3)

['-69.7415041', '-17.74631895', '67.41158818', '-71.91233417', '68.44101913', '-58.8784902', '27.39095208']
['5.7217254', '7.71807615', '79.0660602', '-25.39319369', '-62.49004925', '-22.07052049', '-49.38065519']


In [17]:
# lo que haremos ahora es borrar las ultimas dos columnas y luego agragar las nuevas
comando = 'alter table customers drop column CustomerLocationLatitude'
usuario.ejecutar(comando)

comando = 'alter table customers drop column CustomerLocationLongitude'
usuario.ejecutar(comando)



In [18]:
# veamos que se hayan ejecutado los cambios
comando = 'select * from customers'
df1 = usuario.mostrar_tabla(comando)
df1

,0
0,Customer001
1,Customer002
2,Customer003
3,Customer004
4,Customer005
5,Customer006
6,Customer007


In [19]:
# ahora agregemos las columnas nuevas
comando = 'alter table customers add column CustomerLocationLatitude varchar(50)'
usuario.ejecutar(comando)

comando = 'alter table customers add column CustomerLocationLongitude varchar(50)'
usuario.ejecutar(comando)

comando = 'select * from customers'
df2 = usuario.mostrar_tabla(comando)
df2


,0,1,2
0,Customer001,None,None
1,Customer002,None,None
2,Customer003,None,None
3,Customer004,None,None
4,Customer005,None,None
5,Customer006,None,None
6,Customer007,None,None


In [20]:
# voy a agregar una tercera columna como indice para introducir los valores
comando = 'alter table customers add column indice serial'
usuario.ejecutar(comando)

comando = 'select * from customers'
df3 = usuario.mostrar_tabla(comando)
df3


,0,1,2,3
0,Customer001,None,None,1
1,Customer002,None,None,2
2,Customer003,None,None,3
3,Customer004,None,None,4
4,Customer005,None,None,5
5,Customer006,None,None,6
6,Customer007,None,None,7


In [23]:
# ahora voy a actualizar los valores de las columas con los datos guardados
# modificaciones para columna 2
for ind, valor in enumerate(columna2):
    comando_base = 'update customers set CustomerLocationLatitude = {} where indice = {}'.format(valor, ind+1)
    usuario.ejecutar(comando_base)
    
# modificaciones para columna 3
for ind, valor in enumerate(columna3):
    comando_base = 'update customers set CustomerLocationLongitude = {} where indice = {}'.format(valor, ind+1)
    usuario.ejecutar(comando_base)
    
comando = 'select * from customers'
df4 = usuario.mostrar_tabla(comando)
df4




,0,1,2,3
0,Customer001,-69.7415041,5.7217254,1
1,Customer002,-17.74631895,7.71807615,2
2,Customer003,67.41158818,79.0660602,3
3,Customer004,-71.91233417,-25.39319369,4
4,Customer005,68.44101913,-62.49004925,5
5,Customer006,-58.8784902,-22.07052049,6
6,Customer007,27.39095208,-49.38065519,7


In [24]:
# ahora solo resta borrar la columan que utilizamos de indice
comando = 'alter table customers drop column indice'
usuario.ejecutar(comando)

comando = 'select * from customers'
df5 = usuario.mostrar_tabla(comando)
df5


,0,1,2
0,Customer001,-69.7415041,5.7217254
1,Customer002,-17.74631895,7.71807615
2,Customer003,67.41158818,79.0660602
3,Customer004,-71.91233417,-25.39319369
4,Customer005,68.44101913,-62.49004925
5,Customer006,-58.8784902,-22.07052049
6,Customer007,27.39095208,-49.38065519


## **Notas: Creo que aun se pueden desarrollar ciertas funciones que nos pueden facilitar aun mas todo el proceso anterior.**

### A continuacion, desarrollemos una funcion que nos permita agregar columanas rapidamente a una tabla, y agreguemos a la funcion de mostrar tabla una manera para que tambien de la opcion de mostrar el nombre de las columnas.

In [41]:
# funcion para agregar una columna (formato lista)

def agregar_columna(nombre_tabla, nombre_columna, lista_datos, tipo_columna='varchar(50)'):
    
    # agregar columna
    ## por ahora lo dejare con dos tipos de datos (int y varchar(por defecto))
    if tipo_columna == 'varchar(50)':
        comando_base = 'alter table {} add column {} {}'.format(nombre_tabla, nombre_columna, tipo_columna)
    else:
        comando_base == 'alter table {} add column {} {}'.format(nombre_tabla, nombre_columna, 'int')
    usuario.ejecutar(comando_base)    
    
    # rellenar con los datos
    ## agregar la columna de indice
    comando = 'alter table {} add column indice serial'.format(nombre_tabla)
    usuario.ejecutar(comando)
    
    ## actualizar datos
    for ind, valor in enumerate(lista_datos):
        comando_base = 'update {} set {} = {} where indice = {}'.format(nombre_tabla, nombre_columna, valor, ind+1)
        usuario.ejecutar(comando_base)
        
    ## borrar el indice 
    comando = 'alter table {} drop column indice'.format(nombre_tabla)
    usuario.ejecutar(comando)
            
    

## Tabla DCs

In [32]:
# veamos que tiene
comando = 'select * from DC'
df = usuario.mostrar_tabla(comando)
df


,0,1,2,3,4
0,DC001,-1.167.114.585,-8.322.489.278,88,48
1,DC002,-8.195.947.549,7.291.626.950,37,96
2,DC003,-1.422.731.698,6.443.633.751,58,82
3,DC004,4.358.105.746,8.986.685.836,48,29
4,DC005,8.411.115.264,-3.458.885.675,93,49


In [34]:
# Notamos que la columna 2 y 3, debemos hacer los mismo cambios que en la anterior tabla
columna2 = list(df[1][:])
columna3 = list(df[2][:])

columna2 = [int(valor.replace('.', ''))/100000000 for valor in columna2]
columna3 = [int(valor.replace('.', ''))/100000000 for valor in columna3]

columna2 = [str(valor) for valor in columna2]
columna3 = [str(valor) for valor in columna3]

print(columna2)
print(columna3)

['-11.67114585', '-81.95947549', '-14.22731698', '43.58105746', '84.11115264']
['-83.22489278', '72.9162695', '64.43633751', '89.86685836', '-34.58885675']


In [35]:
# ahora borremos las columnas 2 y 3 de la tabla
comando = 'alter table dc drop column DCLocationLatitude'
usuario.ejecutar(comando)

comando = 'alter table dc drop column DCLocationLongitude'
usuario.ejecutar(comando)

comando = 'select * from DC'
df1 = usuario.mostrar_tabla(comando)
df1


,0,1,2
0,DC001,88,48
1,DC002,37,96
2,DC003,58,82
3,DC004,48,29
4,DC005,93,49


In [44]:
# ahora agreguemos las columnas, probando la nueva funcion
nombre_tabla = 'dc'
nombre_columna = 'DCLocationLatitude'
lista_datos = columna2
agregar_columna(nombre_tabla, nombre_columna, lista_datos)

nombre_tabla = 'dc'
nombre_columna = 'DCLocationLongitude'
lista_datos = columna3
agregar_columna(nombre_tabla, nombre_columna, lista_datos)


comando = 'select * from DC'
df2 = usuario.mostrar_tabla(comando)
df2



,0,1,2,3,4
0,DC001,88,48,-11.67114585,-83.22489278
1,DC002,37,96,-81.95947549,72.9162695
2,DC003,58,82,-14.22731698,64.43633751
3,DC004,48,29,43.58105746,89.86685836
4,DC005,93,49,84.11115264,-34.58885675


In [46]:
# por utimo, verifiquemos que los datos se encuentren en el formato correcto

print(df2[1][0])
print(type(df2[1][0]))

print(df2[2][0])
print(type(df2[2][0]))

## estan correctos

88
<class 'numpy.int64'>
48
<class 'numpy.int64'>


In [1]:
# correccion para mostrar el nombre de las columnas de las tablas
from raaraya1 import SQL
usuario = SQL('curso_postgresql', 'Tester', '123456')

comando = 'select * from dc'
columnas, df = usuario.mostrar_tabla(comando, 'dc')
print(columnas)
df

[('(dcid,"character varying")',), ('(dctransportationcapacityliters,integer)',), ('(handlingcostperliter,integer)',), ('(dclocationlatitude,"character varying")',), ('(dclocationlongitude,"character varying")',)]


,0,1,2,3,4
0,DC001,88,48,-11.67114585,-83.22489278
1,DC002,37,96,-81.95947549,72.9162695
2,DC003,58,82,-14.22731698,64.43633751
3,DC004,48,29,43.58105746,89.86685836
4,DC005,93,49,84.11115264,-34.58885675


## Bien, ahora continuemos con el resto

## Tabla detergents

In [2]:
comando = 'select * from detergents'
columnas, df = usuario.mostrar_tabla(comando, 'detergents')
print(columnas)
df

[('(detergentid,"character varying")',), ('(composition,"character varying")',), ('(puritypercentage,"character varying")',), ('(packaging,"character varying")',), ('(nominalpriceperliter,"character varying")',)]


,0,1,2,3,4
0,DetergentID,Composition,PurityPercentage,Packaging,NominalPricePerLiter
1,Det001,C71,90,crysta500cc,766
2,Det002,C80,95,crysta1L,1989
3,Det003,C75,90,crystal40cc,1077
4,Det004,C79,95,crysta500cc,1140
5,Det005,C79,95,crystal40cc,1702


In [1]:
# Como se puede observar, tiene un error al ingresar los datos en la primera fila 
## creo que esto sucedio cuando la importe la tabla con el administrador (con el nombre de las columnas deshabilitado)
from raaraya1 import SQL
import pandas as pd
import psycopg2

usuario = SQL('curso_postgresql', 'Tester', '123456')

# para borrar esta fila, debemos utilizar el comando delete
comando = 'delete from detergents where detergentid = detergentID'
usuario.ejecutar(comando)

# verifiquemos que se haya borrado el registro
comando = 'select * from detergents'
columnas, df = usuario.mostrar_tabla(comando, 'detergents')
print(columnas)
df



[('(detergentid,"character varying")',), ('(composition,"character varying")',), ('(puritypercentage,"character varying")',), ('(packaging,"character varying")',), ('(nominalpriceperliter,"character varying")',)]


""


## Me acaba de suceder que se borraron todos los registros de la tabla (esto me parece que sucedio, porque puse las comillas en el where)

### voy a borrar la tabla e importarla correctamente esta vez


In [7]:
# borrar la tabla
comando = 'drop table detergents'
usuario.ejecutar(comando)

# generar el dataframe
ruta = 'https://raw.githubusercontent.com/raaraya1/University-Projects/main/Logistics/Datos/detergents.csv'
archivo = pd.read_csv(ruta, sep=';')
df = pd.DataFrame(archivo)

# cargar el dataframe en SQL
usuario.importar('detergents', df)

# mostrar la tabla
comando = 'select * from detergents'
columnas, df = usuario.mostrar_tabla(comando, 'detergents')
print(columnas)
df



[('(detergentid,"character varying")',), ('(composition,"character varying")',), ('(puritypercentage,"character varying")',), ('(packaging,"character varying")',), ('(nominalpriceperliter,"character varying")',)]


,0,1,2,3,4
0,Det001,C71,90,crysta500cc,766
1,Det002,C80,95,crysta1L,1989
2,Det003,C75,90,crystal40cc,1077
3,Det004,C79,95,crysta500cc,1140
4,Det005,C79,95,crystal40cc,1702


### Ahora si estan correctas las filas de la tabla, **lo que resta cambiar es el tipo de dato de la columna 2 y 4**

In [8]:
# cambiar el tipo de dato a entero para las columnas 2 y 4

# columna 2
usuario.column_type_int('detergents', 'puritypercentage')

# columna 4
usuario.column_type_int('detergents', 'nominalpriceperliter')

# verificar cambios mostrando la tabla
comando = 'select * from detergents'
columnas, df = usuario.mostrar_tabla(comando, 'detergents')
print(columnas)
df


[('(detergentid,"character varying")',), ('(composition,"character varying")',), ('(packaging,"character varying")',), ('(puritypercentage,integer)',), ('(nominalpriceperliter,integer)',)]


,0,1,2,3,4
0,Det001,C71,crysta500cc,90,766
1,Det002,C80,crysta1L,95,1989
2,Det003,C75,crystal40cc,90,1077
3,Det004,C79,crysta500cc,95,1140
4,Det005,C79,crystal40cc,95,1702


## Tabla orders


In [9]:
# mostrar la tabla
comando = 'select * from orders'
columnas, df = usuario.mostrar_tabla(comando, 'orders')
print(columnas)
df


[('(orderid,"character varying")',), ('(productid,"character varying")',), ('(customerid,"character varying")',), ('(quantityliters,integer)',)]


,0,1,2,3
0,Order001,Det004,Customer001,9
1,Order002,Det002,Customer007,19
2,Order003,Det001,Customer003,19
3,Order004,Det002,Customer005,16
4,Order005,Det005,Customer002,2
5,Order006,Det003,Customer007,5
6,Order007,Det003,Customer002,1
7,Order008,Det003,Customer005,16
8,Order009,Det005,Customer007,9
9,Order0010,Det001,Customer002,13


### No necesita modificaciones

## Tabla plants


In [4]:
# mostrar la tabla
comando = 'select * from plants'
columnas, df = usuario.mostrar_tabla(comando, 'plants')
print(columnas)
df


[('(plantid,"character varying")',), ('(plantlocationlatitude,"character varying")',), ('(plantlocationlongitude,"character varying")',), ('(plantprodcapacityliters,"character varying")',), ('(plantrailtransportationcapacityliters,"character varying")',), ('(plantroadtransportationcapacityliters,"character varying")',)]


,0,1,2,3,4,5
0,Plant001,6.795.458.019,3.095.143.418,760,324,362
1,Plant002,1.838.170.216,-2.386.330.645,906,76,201
2,Plant003,6.177.910.668,-2.966.737.339,691,382,345
3,Plant004,7.866.031.164,8.144.016.527,165,294,445
4,Plant005,5.305.397.278,-9.524.078.678,1000,292,499


### Necesita correciones en las dos primeras filas y cambiar el tipo de dato de las ultimas 3 columnas

In [7]:
# guardar las columnas
columna1 = list(df[1][:])
columna2 = list(df[2][:])

columna1 = [valor.replace('.', '') for valor in columna1]
columna2 = [valor.replace('.', '') for valor in columna2]

columna1 = [float(valor)/100000000 for valor in columna1]
columna2 = [float(valor)/100000000 for valor in columna2]

columna1 = [str(valor) for valor in columna1]
columna2 = [str(valor) for valor in columna2]

print(columna1)
print(columna2)



ValueError: invalid literal for int() with base 10: '67.95458019'

In [2]:
from raaraya1 import SQL
import pandas as pd
import psycopg2

usuario = SQL('curso_postgresql', 'Tester', '123456')


In [6]:
# primero eliminamos las columnas 1 y 2
comando = 'alter table plants drop column plantlocationlatitude'
usuario.ejecutar(comando)

comando = 'alter table plants drop column plantlocationlongitude'
usuario.ejecutar(comando)

# mostrar la tabla
comando = 'select * from plants'
columnas1, df1 = usuario.mostrar_tabla(comando, 'plants')
print(columnas1)
df1



[('(plantid,"character varying")',), ('(plantprodcapacityliters,"character varying")',), ('(plantrailtransportationcapacityliters,"character varying")',), ('(plantroadtransportationcapacityliters,"character varying")',)]


,0,1,2,3
0,Plant001,760,324,362
1,Plant002,906,76,201
2,Plant003,691,382,345
3,Plant004,165,294,445
4,Plant005,1000,292,499


In [8]:
# ahora agregamos las columnas que guardamos

usuario.agregar_columna('plants', 'plantlocationlatitude', columna1, tipo_columna='varchar(50)')
usuario.agregar_columna('plants', 'plantlocationlongitude', columna2, tipo_columna='varchar(50)')


# mostrar la tabla
comando = 'select * from plants'
columnas2, df2 = usuario.mostrar_tabla(comando, 'plants')
print(columnas2)
df2


[('(plantid,"character varying")',), ('(plantprodcapacityliters,"character varying")',), ('(plantrailtransportationcapacityliters,"character varying")',), ('(plantroadtransportationcapacityliters,"character varying")',), ('(plantlocationlatitude,"character varying")',), ('(plantlocationlongitude,"character varying")',)]


,0,1,2,3,4,5
0,Plant001,760,324,362,67.95458019,30.95143418
1,Plant002,906,76,201,18.38170216,-23.86330645
2,Plant003,691,382,345,61.77910668,-29.66737339
3,Plant004,165,294,445,78.66031164,81.44016527
4,Plant005,1000,292,499,53.05397278,-95.24078678


### ahora toca cambiar le tipo de las columnas restantes

In [9]:
# cambiar tipo
usuario.column_type_int('plants', 'plantprodcapacityliters')
usuario.column_type_int('plants', 'plantrailtransportationcapacityliters')
usuario.column_type_int('plants', 'plantroadtransportationcapacityliters')

# mostrar la tabla
comando = 'select * from plants'
columnas3, df3 = usuario.mostrar_tabla(comando, 'plants')
print(columnas3)
df3


[('(plantid,"character varying")',), ('(plantlocationlatitude,"character varying")',), ('(plantlocationlongitude,"character varying")',), ('(plantprodcapacityliters,integer)',), ('(plantrailtransportationcapacityliters,integer)',), ('(plantroadtransportationcapacityliters,integer)',)]


,0,1,2,3,4,5
0,Plant001,67.95458019,30.95143418,760,324,362
1,Plant002,18.38170216,-23.86330645,906,76,201
2,Plant003,61.77910668,-29.66737339,691,382,345
3,Plant004,78.66031164,81.44016527,165,294,445
4,Plant005,53.05397278,-95.24078678,1000,292,499


## Tabla routes_available


In [10]:
# mostrar la tabla
comando = 'select * from routes_available'
columnas, df = usuario.mostrar_tabla(comando, 'routes_available')
print(columnas)
df


[('(routeid,"character varying")',), ('(startroute,"character varying")',), ('(middleroute,"character varying")',), ('(endroute,"character varying")',), ('(railuse,integer)',)]


,0,1,2,3,4
0,Route0097,Plant001,DC002,Customer005,0
1,Route0098,Plant001,DC005,Customer005,0
2,Route0099,Plant001,DC002,Customer005,1
3,Route00100,Plant002,DC001,Customer006,1
4,Route00101,Plant002,DC003,Customer006,1
...,...,...,...,...,...
115,Route00116,Plant001,nan,Customer007,0
116,Route00117,Plant002,nan,Customer007,1
117,Route00118,Plant004,DC001,Customer002,0
118,Route00119,Plant001,DC001,Customer003,0


### **No necesita realizarse cambios**

## Tabla routes_endtoend

In [11]:
# mostrar la tabla
comando = 'select * from routes_endtoend'
columnas, df = usuario.mostrar_tabla(comando, 'routes_endtoend')
print(columnas)
df


UndefinedTable: no existe la relación «routes_endtoend»
LINE 1: select * from routes_endtoend
                      ^


### JAJA, Me acabo de dar cuenta que no cargue esta tabla, hagasmoslo rapidamente (**recordar que esta es la tabla que tiene el problema con el nombre de columna 'End'**)


In [22]:
# generar el dataframe
ruta = 'https://raw.githubusercontent.com/raaraya1/University-Projects/main/Logistics/Datos/routes_endtoend.csv'
archivo = pd.read_csv(ruta, sep=';')
df = pd.DataFrame(archivo)

# corregimos el error con el nombre de la columna
df['Final'] = df['End']
df = df[['Start', 'Final', 'EndToEndCostPerLiter']]

# cargar el dataframe en SQL
usuario.importar('routes_endtoend', df)

# mostrar la tabla
comando = 'select * from routes_endtoend'
columnas, df = usuario.mostrar_tabla(comando, 'routes_endtoend')
print(columnas)
df



[('(start,"character varying")',), ('(final,"character varying")',), ('(endtoendcostperliter,"character varying")',)]


,0,1,2
0,Plant001,Customer001,9
1,Plant002,Customer001,7
2,Plant002,Customer002,9
3,Plant003,Customer002,5
4,Plant004,Customer002,6
5,Plant001,Customer002,7
6,Plant004,Customer003,3
7,Plant001,Customer003,10
8,Plant002,Customer003,7
9,Plant003,Customer003,6


In [23]:
# Ahora debemos cambiar el tipo de la ultima columna
usuario.column_type_int('routes_endtoend', 'endtoendcostperliter')    

# mostrar la tabla
comando = 'select * from routes_endtoend'
columnas, df = usuario.mostrar_tabla(comando, 'routes_endtoend')
print(columnas)
df


[('(start,"character varying")',), ('(final,"character varying")',), ('(endtoendcostperliter,integer)',)]


,0,1,2
0,Plant001,Customer001,9
1,Plant002,Customer001,7
2,Plant002,Customer002,9
3,Plant003,Customer002,5
4,Plant004,Customer002,6
5,Plant001,Customer002,7
6,Plant004,Customer003,3
7,Plant001,Customer003,10
8,Plant002,Customer003,7
9,Plant003,Customer003,6


## Tabla routes_segment


In [24]:
# mostrar la tabla
comando = 'select * from routes_segment'
columnas, df = usuario.mostrar_tabla(comando, 'routes_segment')
print(columnas)
df


[('(segmentstart,"character varying")',), ('(segmentend,"character varying")',), ('(segmentmode,"character varying")',), ('(segmentcostperunit,"character varying")',)]


,0,1,2,3
0,Plant001,DC001,Road,4
1,Plant002,DC002,Rail,4
2,Plant001,DC004,Road,6
3,Plant001,DC004,Rail,3
4,Plant002,DC002,Road,5
...,...,...,...,...
115,Plant002,Customer007,Road,9
116,Plant001,Customer002,Rail,1
117,Plant003,Customer002,Road,9
118,Plant004,Customer002,Rail,7


In [25]:
# Solo hace falta cambiar el tipo de la ultima columna
usuario.column_type_int('routes_segment', 'segmentcostperunit')    

# mostrar la tabla
comando = 'select * from routes_segment'
columnas, df = usuario.mostrar_tabla(comando, 'routes_segment')
print(columnas)
df


[('(segmentstart,"character varying")',), ('(segmentend,"character varying")',), ('(segmentmode,"character varying")',), ('(segmentcostperunit,integer)',)]


,0,1,2,3
0,Plant001,DC001,Road,4
1,Plant002,DC002,Rail,4
2,Plant001,DC004,Road,6
3,Plant001,DC004,Rail,3
4,Plant002,DC002,Road,5
...,...,...,...,...
115,Plant002,Customer007,Road,9
116,Plant001,Customer002,Rail,1
117,Plant003,Customer002,Road,9
118,Plant004,Customer002,Rail,7


## Tabla variable_costs

In [26]:
# mostrar la tabla
comando = 'select * from variable_costs'
columnas, df = usuario.mostrar_tabla(comando, 'variable_costs')
print(columnas)
df


[('(plantid,"character varying")',), ('(detergentid,"character varying")',), ('(costperliter,"character varying")',)]


,0,1,2
0,Plant001,Det001,20
1,Plant002,Det001,20
2,Plant003,Det001,20
3,Plant004,Det001,20
4,Plant005,Det001,20
5,Plant001,Det002,20
6,Plant002,Det002,20
7,Plant003,Det002,20
8,Plant004,Det002,20
9,Plant005,Det002,20


In [27]:
# Solo hace falta cambiar la ultima columna
usuario.column_type_int('variable_costs', 'costperliter')    

# mostrar la tabla
comando = 'select * from variable_costs'
columnas, df = usuario.mostrar_tabla(comando, 'variable_costs')
print(columnas)
df


[('(plantid,"character varying")',), ('(detergentid,"character varying")',), ('(costperliter,integer)',)]


,0,1,2
0,Plant001,Det001,20
1,Plant002,Det001,20
2,Plant003,Det001,20
3,Plant004,Det001,20
4,Plant005,Det001,20
5,Plant001,Det002,20
6,Plant002,Det002,20
7,Plant003,Det002,20
8,Plant004,Det002,20
9,Plant005,Det002,20


# TODAS LAS TABLAS ARREGLADAS
## **En el siguiente cuadernillo, lo que se hará es establecer las claves primarias y foraneas, formar las relaciones y guardar vistas**